In [103]:
import pandas as pd
import itertools
import numpy as np

In [113]:
df = pd.read_csv("fb.csv")

to_search = df[(df["average log2 error"] < 15) & (df["size binary search"] < 2000000000)].sort_values("average log2 error").copy()

# if the best non-radix model has an error worse than 5, use radix. Otherwise, don't.
if to_search[~to_search.layers.str.contains("radix")].sort_values("average log2 error").iloc[0]["average log2 error"] < 10:
    to_search = to_search[~to_search.layers.str.contains("radix")].copy()

In [114]:
front = to_search[to_search.front & (to_search["average log2 error"] < 15) & (to_search["size binary search"] < 2000000000)].copy()
front.sort_values("average log2 error")

,layers,branching factor,average error,average error %,average l2 error,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front
126,"linear_spline,linear",2097152,127.140537,0.001914,4.226717e+07,7.016945,3828,0.001914,50331664,33554448,nm77680171b4d11b70aa2f,True,False,True
133,"linear_spline,linear",1048576,183.943797,0.002550,9.123860e+07,7.723291,5099,0.002550,25165840,16777232,nm9d7314ecfb0606fea357,True,False,True
7,"linear_spline,linear",524288,266.117007,0.002589,2.087363e+08,8.428844,5178,0.002589,12582928,8388624,nm14697a215f79906b7b7d,True,False,True
143,"linear_spline,linear",262144,383.549167,0.002908,5.037689e+08,9.120617,5816,0.002908,6291472,4194320,nmd985905d60efa476204b,True,False,True
118,"linear,linear",131072,554.474509,0.005482,1.338033e+09,9.793821,10964,0.005482,3145744,2097168,nm4ba04214fe8ae1c6a0ef,True,False,True
39,"linear_spline,linear",65536,799.587735,0.003387,3.846815e+09,10.430103,6774,0.003387,1572880,1048592,nm6d2f6a0dda092b70154a,True,False,True
113,"linear_spline,linear",32768,1151.260790,0.004347,1.228171e+10,11.030974,8695,0.004347,786448,524304,nm238bd4f690b492c7d5e8,True,False,True
139,"linear_spline,linear",16384,1664.141578,0.004609,4.346324e+10,11.609133,9218,0.004609,393232,262160,nmc64eb7f62650bbe7dfff,True,False,True
69,"linear_spline,linear",8192,2413.632627,0.004276,1.666377e+11,12.167599,8551,0.004276,196624,131088,nmaac4a789ceba69493dce,True,False,True
122,"linear_spline,linear",4096,3499.589229,0.004722,6.688541e+11,12.716431,9443,0.004722,98320,65552,nm6502100e724c28edaaf0,True,False,True


In [115]:
front = front.to_dict("record")

In [116]:
retained = []
for rmi1 in front:
    for rmi2 in front:
        if rmi1["layers"] == rmi2["layers"] and rmi1["branching factor"] == rmi2["branching factor"]:
            continue
        
        if abs(rmi1["average log2 error"] - rmi2["average log2 error"]) < 0.001:
            if rmi1["size binary search"] > rmi2["size binary search"]:
                break
                
        if abs(rmi1["size binary search"] - rmi2["size binary search"]) < 1000:
            if rmi1["average log2 error"] > rmi2["average log2 error"]:
                break
    else:
        retained.append(rmi1)
        
retained = pd.DataFrame(retained).sort_values("average log2 error")
retained

,layers,branching factor,average error,average error %,average l2 error,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front
0,"linear_spline,linear",2097152,127.140537,0.001914,4.226717e+07,7.016945,3828,0.001914,50331664,33554448,nm77680171b4d11b70aa2f,True,False,True
1,"linear_spline,linear",1048576,183.943797,0.002550,9.123860e+07,7.723291,5099,0.002550,25165840,16777232,nm9d7314ecfb0606fea357,True,False,True
2,"linear_spline,linear",524288,266.117007,0.002589,2.087363e+08,8.428844,5178,0.002589,12582928,8388624,nm14697a215f79906b7b7d,True,False,True
3,"linear_spline,linear",262144,383.549167,0.002908,5.037689e+08,9.120617,5816,0.002908,6291472,4194320,nmd985905d60efa476204b,True,False,True
4,"linear,linear",131072,554.474509,0.005482,1.338033e+09,9.793821,10964,0.005482,3145744,2097168,nm4ba04214fe8ae1c6a0ef,True,False,True
5,"linear_spline,linear",65536,799.587735,0.003387,3.846815e+09,10.430103,6774,0.003387,1572880,1048592,nm6d2f6a0dda092b70154a,True,False,True
6,"linear_spline,linear",32768,1151.260790,0.004347,1.228171e+10,11.030974,8695,0.004347,786448,524304,nm238bd4f690b492c7d5e8,True,False,True
7,"linear_spline,linear",16384,1664.141578,0.004609,4.346324e+10,11.609133,9218,0.004609,393232,262160,nmc64eb7f62650bbe7dfff,True,False,True
8,"linear_spline,linear",8192,2413.632627,0.004276,1.666377e+11,12.167599,8551,0.004276,196624,131088,nmaac4a789ceba69493dce,True,False,True
9,"linear_spline,linear",4096,3499.589229,0.004722,6.688541e+11,12.716431,9443,0.004722,98320,65552,nm6502100e724c28edaaf0,True,False,True


In [117]:
retained["diff"] = [-1_000_000_000_000] + list(np.diff(retained.sort_values("size binary search", ascending=False)["size binary search"].to_numpy()))
selected = retained.sort_values("diff", ascending=True)[0:10]
selected = selected.sort_values("size binary search", ascending=False)
selected

,layers,branching factor,average error,average error %,average l2 error,average log2 error,max error,max error %,size binary search,size linear search,namespace,binary,star,front,diff
0,"linear_spline,linear",2097152,127.140537,0.001914,4.226717e+07,7.016945,3828,0.001914,50331664,33554448,nm77680171b4d11b70aa2f,True,False,True,-1000000000000
1,"linear_spline,linear",1048576,183.943797,0.002550,9.123860e+07,7.723291,5099,0.002550,25165840,16777232,nm9d7314ecfb0606fea357,True,False,True,-25165824
2,"linear_spline,linear",524288,266.117007,0.002589,2.087363e+08,8.428844,5178,0.002589,12582928,8388624,nm14697a215f79906b7b7d,True,False,True,-12582912
3,"linear_spline,linear",262144,383.549167,0.002908,5.037689e+08,9.120617,5816,0.002908,6291472,4194320,nmd985905d60efa476204b,True,False,True,-6291456
4,"linear,linear",131072,554.474509,0.005482,1.338033e+09,9.793821,10964,0.005482,3145744,2097168,nm4ba04214fe8ae1c6a0ef,True,False,True,-3145728
5,"linear_spline,linear",65536,799.587735,0.003387,3.846815e+09,10.430103,6774,0.003387,1572880,1048592,nm6d2f6a0dda092b70154a,True,False,True,-1572864
6,"linear_spline,linear",32768,1151.260790,0.004347,1.228171e+10,11.030974,8695,0.004347,786448,524304,nm238bd4f690b492c7d5e8,True,False,True,-786432
7,"linear_spline,linear",16384,1664.141578,0.004609,4.346324e+10,11.609133,9218,0.004609,393232,262160,nmc64eb7f62650bbe7dfff,True,False,True,-393216
8,"linear_spline,linear",8192,2413.632627,0.004276,1.666377e+11,12.167599,8551,0.004276,196624,131088,nmaac4a789ceba69493dce,True,False,True,-196608
9,"linear_spline,linear",4096,3499.589229,0.004722,6.688541e+11,12.716431,9443,0.004722,98320,65552,nm6502100e724c28edaaf0,True,False,True,-98304


In [118]:
DATASET = "fb_200M_uint64"
tmp = selected.reset_index().drop(columns="index").reset_index()
tmp["namespace"] = tmp.apply(lambda x: DATASET + "_" + str(x["index"]), axis=1)
spec = {"configs" : tmp[["layers", "branching factor", "namespace", "binary"]].to_dict("record")}

import json
print(json.dumps(spec))

{"configs": [{"layers": "linear_spline,linear", "branching factor": 2097152, "namespace": "fb_200M_uint64_0", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 1048576, "namespace": "fb_200M_uint64_1", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 524288, "namespace": "fb_200M_uint64_2", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 262144, "namespace": "fb_200M_uint64_3", "binary": true}, {"layers": "linear,linear", "branching factor": 131072, "namespace": "fb_200M_uint64_4", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 65536, "namespace": "fb_200M_uint64_5", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 32768, "namespace": "fb_200M_uint64_6", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 16384, "namespace": "fb_200M_uint64_7", "binary": true}, {"layers": "linear_spline,linear", "branching factor": 8192, "namespace": "fb_200M_uint64_8